[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). All rights reserved.

# DIY Covid-19 Dashboard

In [87]:
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

from uk_covid19 import Cov19API

from datetime import date
from datetime import datetime

In [80]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

## Load initial data from disk


In [82]:
# Load JSON files and store the raw data in some variable. Edit as appropriate:

# Comparing different testing levels (incl. total) in UK:

filters = [
    'areaType=overview',
]

structure = {
    "date": "date",
    "PillarOneTesting":"cumPillarOneTestsByPublishDate",
    "PillarTwoTesting":"cumPillarTwoTestsByPublishDate",
    "PillarThreeTesting":"cumPillarThreeTestsByPublishDate",
    "PillarFourTesting":"cumPillarFourTestsByPublishDate",
    "Total":"cumTestsByPublishDate"
}

api = Cov19API(filters=filters, structure=structure)

jsondata=api.get_json()

with open("testingseries.json", "wt") as OUTF:
    json.dump(jsondata, OUTF)
    

## Wrangle the data


In [83]:
def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")

def wrangle_data(jsondata):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Edit to include the code that wrangles the data, creates the dataframe and fills it in. """
    
    # Actual data is stored as a list of dictionaries under the data key, retrieveing it:
    datalist=jsondata['data']
    # Extracting all the dates and sorting them. 
    dates=[dictionary['date'] for dictionary in datalist]
    # With dates written year-first, alphabetical ordering does the trick:
    dates.sort()
    
    # Finding the earliest and latest date and converting them to the pandas type for representing dates:
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    
    # Creating an index as a date_range - the date analog of a range for integers, and it will include any dates that may be missing from our list:
    index=pd.date_range(startdate, enddate, freq='D') # 'D' - for day
    # Defining the DateFrame object by specifying its index and the title of its columns:
    df=pd.DataFrame(index=index, columns=['PillarOneTesting', 'PillarTwoTesting', 'PillarThreeTesting', 'PillarFourTesting', 'Total']) # Constructor 
    
    # Filling the DataFrame with our data:
    for entry in datalist: # Each entry is a dictionary with date, cases, hospital and deaths
        date=parse_date(entry['date'])
        for column in ['PillarOneTesting', 'PillarTwoTesting', 'PillarThreeTesting', 'PillarFourTesting', 'Total']:
        # Check that nothing is there yet - just in case some dates are duplicated, 
        # maybe with data for different columns in each entry:
            if pd.isna(df.loc[date, column]): 
                # To detect missing values; isna = is None. 
                # Replacing None with 0 in our data:
                value=float(entry[column]) if entry[column]!=None else 0.0
                # Accessing a specific location in the dataframe - using df.loc
                # and inputting - index, column - in a single set of [ ]:
                df.loc[date, column]=value
            
    # Filling in any remaining "holes" due to missing dates:
    df.fillna(0.0, inplace=True)
    
    return df

# Putting the wrangling code into a function allows you to call it again after refreshing the data through 
# the API. You should call the function directly on the JSON data when the dashboard starts, by including 
# the call in the cell as below:
testingseriesdf=wrangle_data(jsondata) # ...df is the dataframe for plotting.

testingseriesdf


,PillarOneTesting,PillarTwoTesting,PillarThreeTesting,PillarFourTesting,Total
2020-03-31,153588.0,1586.0,0.0,0.0,155174.0
2020-04-01,164844.0,2254.0,0.0,139.0,167237.0
2020-04-02,179190.0,2857.0,0.0,305.0,182352.0
2020-04-03,219403.0,3534.0,0.0,641.0,223578.0
2020-04-04,234506.0,4330.0,0.0,822.0,239658.0
...,...,...,...,...,...
2020-11-19,11656202.0,23255669.0,1833205.0,3521107.0,40266183.0
2020-11-20,11736070.0,23530373.0,1836157.0,3559545.0,40662145.0
2020-11-21,11815788.0,23780874.0,1837835.0,3598405.0,41032902.0
2020-11-22,11880462.0,23953113.0,1838375.0,3644864.0,41316814.0


## Download current data

Give your users an option to refresh the dataset - a "refresh" button will do. The button callback should
* call the code that accesses the API and download some fresh raw data;
* wrangle that data into a dataframe and update the corresponding (global) variable for plotting;
* optionally: force a redraw of the graph and give the user some fredback.

Once you get it to work, you may want to wrap your API call inside an exception handler, so that the user is informed, the "canned" data are not overwritten and nothing crashes if for any reason the server cannot be reached or data are not available.

After you refresh the data, graphs will not update until the user interacts with a widget. You can trick ```iPywidgets``` into redrawing the graph by simulating interaction, as in the ```refresh_graph``` function we define in the Graph and Analysis section below.

Clicking on the button below just generates some more random data and refreshes the graph. The button should read *Fetch Data*. If you see anything else, take a deep breath :)

In [84]:
# Place your API access code in this function. Do not call this function directly; it will be called by 
# the button callback. 
def access_api():
    """ Accesses the PHE API. Returns raw data in the same format as data loaded from the "canned" JSON file. """
    filters = [
        'areaType=overview',
    ]

    structure = {
        "date": "date",
        "PillarOneTesting":"cumPillarOneTestsByPublishDate",
        "PillarTwoTesting":"cumPillarTwoTestsByPublishDate",
        "PillarThreeTesting":"cumPillarThreeTestsByPublishDate",
        "PillarFourTesting":"cumPillarFourTestsByPublishDate",
        "Total":"cumTestsByPublishDate"
    }
    
    api = Cov19API(filters=filters, structure=structure)
    jsondata=api.get_json()

    #with open("testingseries.json", "wt") as OUTF:
        #json.dump(jsondata, OUTF)
    
    #with open("testingseries.json", "rt") as INF:
        #jsondata=json.load(INF)
    
    return jsondata 

In [103]:
# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button:
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Getting fresh data from the API. If you have time, include some error handling - via except - 
    # around this call:
    
    def refresh_graph():
        """ We change the value of the widget in order to force a redraw of the graph;
        this is useful when the data have been updated. This is a bit of a gimmick; it
        needs to be customised for one of your widgets. """
        current=scale.value
        if current==scale.options[0]:
            other=scale.options[1]
        else:
            other=scale.options[0]
        scale.value=other # forces the redraw
        scale.value=current # now we can change it backdef refresh_graph():
        
    try: 
        apidata=access_api()
        # Wrangling the data and overwriting the dataframe for plotting:
        global df
        df=wrangle_data(apidata)
        # The graph won't refresh until the user interacts with the widget.
        # this function simulates the interaction, see Graph and Analysis below.
        # you can omit this step in the first instance
        refresh_graph()
        # After all is done, you can switch the icon on the button to a "check" sign
        # and optionally disable the button - it won't be needed again. You can use icons
        # "unlink" or "times" and change the button text to "Unavailable" in case the 
        # API call fails:
        apibutton.icon="check"
        apibutton.disabled=True
        
        # Show time stamp:
        today=date.today()
        d1=today.strftime("%B %d, %Y")
        now=datetime.now()
        d2=now.strftime("%H:%M:%S")
        print(f'Data last refreshed at {d2} on {d1}.', end='\r', flush=True)
    
    except:
        # apibutton.description="Error"
        # apibutton.button_style='danger'
        # apibutton.icon='exclamation-triangle'
        print('Refresh failed. Try again later.', end='\r', flush=True)
    

## Graphs and Analysis

Include at least one graph with interactive controls, as well as some instructions for the user and/or comments on what the graph represents and how it should be explored (this example shows two random walks)

In [106]:
series=wdg.SelectMultiple(
    options=['PillarOneTesting', 'PillarTwoTesting', 'PillarThreeTesting', 'PillarFourTesting', 'Total'],
    value=['PillarOneTesting', 'PillarTwoTesting', 'PillarThreeTesting', 'PillarFourTesting'],
    rows=5,
    description='Tests:',
    disabled=False
)

scale=wdg.RadioButtons(
    options=['linear', 'log'],
    description='Scale:',
    disabled=False
)

def series_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0: # Error check
        testingseriesdf[list(gcols)].plot(logy=logscale, title='Number of COVID-19 tests processed in the UK')
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
        
apibutton=wdg.Button(
    description="Refresh",
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Click to download current Public Health England data!",
    # FontAwesome names without the `fa-` prefix - try "download"
    icon='refresh' # e.g. exclamation-triangle
)

# Remember to register your button callback function with the button:
apibutton.on_click(api_button_callback) # The name of your function inside these brackets.
        
# Keep calling series_graph(gcols=value_of_series, gscale=value_of_scale); capture output in variable graph   
graph=wdg.interactive_output(series_graph, {'gcols': series, 'gscale': scale})
ctrls=wdg.VBox([series, scale]) # stacked
form=wdg.HBox([graph, ctrls]) # side-by-side

display(form, apibutton)

Button(button_style='success', description='Refresh', icon='refresh', style=ButtonStyle(), tooltip='Click to d…

## Description of different Testing methods:


## Deploying the dashboard

Once your code is ready and you are satisfied with the appearance of the graphs, replace all the text boxes above with the explanations you would like a dashboard user to see. The next step is deploying the dashboard online - there are several [options](https://voila.readthedocs.io/en/stable/deploy.html) for this, we suggest deploying as a [Binder](https://mybinder.org/). This is basically the same technique that has been used to package this tutorial and to deploy this template dashboard. The instructions may seem a bit involved, but the actual steps are surprisingly easy - we will be going through them together during a live session. You will need an account on [GitHub](https://github.com/) for this - if you don't have one already, now it's the time to create it. 

**Author and Copyright Notice** Remember if you deploy this dashboard as a Binder it will be publicly accessible. Take credit for your work! Also acknowledge the data source: *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*